In [1]:
import sqlite3
import pandas as pd

%cd /content/importforecasting_group2/

# Step 1: Create a connection to the SQLite database
conn = sqlite3.connect('economic_data.db')
cursor = conn.cursor()

# Step 2: Fetch the series names and descriptions
cursor.execute('SELECT id, name, desc FROM series')
series_data = cursor.fetchall()

# Create a mapping of series id to (name, desc)
series_map = {row[0]: (row[1], row[2]) for row in series_data}

# Step 3: Fetch the economic indicators (date, series, value)
cursor.execute('''
    SELECT ei.date, ei.series, ei.value
    FROM economic_indicators ei
    JOIN series s ON ei.series = s.id
    ORDER BY ei.date, ei.series;
''')

economic_indicators = cursor.fetchall()

# Step 4: Convert the fetched data into a pandas DataFrame
data = []

for date, series_id, value in economic_indicators:
    name, desc = series_map[series_id]
    data.append({'date': date, 'series': name, 'value': value})

# Step 5: Create a DataFrame and pivot it
econ_indicators_long = pd.DataFrame(data)

# Pivot the DataFrame so that the series names are columns
econ_indicators = econ_indicators_long.pivot(index='date', columns='series', values='value')

# Step 6: Convert the index to datetime if needed
econ_indicators.index = pd.to_datetime(econ_indicators.index)

# Step 7: Close the connection
conn.close()

# Step 8: Display the reconstructed DataFrame
econ_indicators.head()

/content/importforecasting_group2


series,customs_receipts,fedfunds,gdp,imports
date,,,,
1959-01-01,952.0,2.57,7.9,115.440
1959-04-01,1076.0,3.08,9.3,121.057
1959-07-01,1092.0,3.58,0.3,122.212
1959-10-01,1076.0,3.99,1.1,119.109
1960-01-01,1176.0,3.93,9.3,123.608


In [3]:
try:
    # Calculate the weighted average tariff
    econ_indicators['watr'] = econ_indicators['customs_receipts'] / econ_indicators['imports']
except Exception as e:
  print(f"An error occurred: {e}")

# Display the updated DataFrame
print(econ_indicators.head())


series      customs_receipts  fedfunds  gdp  imports      watr
date                                                          
1959-01-01             952.0      2.57  7.9  115.440  8.246708
1959-04-01            1076.0      3.08  9.3  121.057  8.888375
1959-07-01            1092.0      3.58  0.3  122.212  8.935293
1959-10-01            1076.0      3.99  1.1  119.109  9.033742
1960-01-01            1176.0      3.93  9.3  123.608  9.513947
